In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# Load dataset
data = pd.read_csv("ionosphere.data.csv")

# Separate features and target
X = data.drop('Class', axis="columns")
y = data['Class']

# Encode the target variable
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train a RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Step 4: Choose an instance to explain
instance_to_explain = X_test.iloc[0].values  # Convert to NumPy array for consistency
print("Instance to explain:", instance_to_explain)

# Step 5: Generate perturbed samples
def generate_perturbations(instance, num_samples=100):
    perturbations = []
    for _ in range(num_samples):
        noise = np.random.normal(0, 0.5, size=instance.shape)
        perturbed_instance = instance + noise
        perturbations.append(perturbed_instance)
    return np.array(perturbations)

perturbed_data = generate_perturbations(instance_to_explain)

# Step 6: Get predictions for the perturbed data
perturbed_predictions = rf.predict_proba(perturbed_data)

# Step 7: Calculate weights based on distance to original instance (Euclidean distance)
def calculate_weights(original_instance, perturbed_data):
    distances = np.linalg.norm(perturbed_data - original_instance, axis=1)
    return np.exp(-distances)

weights = calculate_weights(instance_to_explain, perturbed_data)

# Step 8: Fit a simple linear regression model
y_perturbed = perturbed_predictions[:, np.argmax(rf.predict_proba([instance_to_explain])[0])]
lr = LinearRegression()
lr.fit(perturbed_data, y_perturbed, sample_weight=weights)

# Step 9: Print the explanation (Feature importance)
print("\nFeature importance (coefficients):")
for i, feature_name in enumerate(X.columns):
    print(f"{feature_name}: {lr.coef_[i]:.4f}")

 

Instance to explain: [ 1.       0.       0.4709   0.22751  0.42328  0.33598  0.25661  0.47619
  0.01852  0.49471 -0.02116  0.53968 -0.34127  0.31217 -0.4127   0.3254
 -0.51587  0.06878 -0.5     -0.1164  -0.14815 -0.1455  -0.14815 -0.38095
 -0.2328   0.00265  0.03574 -0.31739  0.15873 -0.21693  0.24868 -0.24339
  0.2672   0.04233]

Feature importance (coefficients):
A: 0.0950
B: 0.0033
C: 0.1624
D: 0.0514
E: 0.1433
F: 0.0305
G: 0.0797
H: -0.0023
I: 0.0467
J: 0.0077
K: -0.0483
L: 0.0252
M: 0.0095
N: 0.0360
O: 0.0187
P: -0.0343
Q: -0.0121
R: -0.0525
S: 0.0049
T: 0.0381
U: 0.0269
V: -0.0532
W: 0.0152
X: -0.0191
Y: 0.0405
Z: -0.0021
AA: -0.0404
AB: 0.0045
AC: 0.0121
AD: -0.0084
AE: -0.0144
AF: 0.0131
AG: -0.0342
AH: -0.0293


C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
C:\Users\hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
